In [1]:
import psycopg2
import pandas as pd

import types

import re

import numpy as np
import nltk
from nltk.corpus import stopwords
import nltk.tokenize as nt
from nltk.tokenize import RegexpTokenizer

from textblob import blob, Blobber, TextBlob, Sentence, Word, WordList, tokenizers, sentiments, taggers, parsers, classifiers
#from textblob_aptagger import PerceptronTagger

In [2]:
con = psycopg2.connect(dbname="skillsdb",host="dw-instance.cbrlhmbtfrqg.eu-west-2.redshift.amazonaws.com"
                ,port="5439",user="masteruser", password="Ehgh1363")
curs = con.cursor()

In [3]:
pd.read_sql_query('''select count(distinct user_id) from cv''',con)

,count
0,5049


In [4]:
data = pd.read_sql_query('''select distinct cv.user_id, cv_section_attribute.name,
cv.value_char, cv.value_timestamp from cv_section_attribute 
left join cv on cv_section_attribute.id=cv.cv_section_attribute_id''',con)

In [5]:
# data['value_char'] = data['value_char'].map(lambda x: x.strip() if pd.notnull(x) else x)

In [6]:
data.head()

,user_id,name,value_char,value_timestamp
0,3.257237e+11,name,Freelance Copy Editor of 100,NaT
1,1.987262e+11,admit_year,None,2016-01-01
2,1.987262e+11,grad_year,present,NaT
3,1.987262e+11,admit_year,None,2015-01-01
4,1.987262e+11,admit_year,None,2012-01-01


In [7]:
data_valid = data[data['name'].isin(['locale','name','summary','headline',
                                     'degree','school','admit_year','grad_year',
                                     'company',  'title',  'work_location',  
                                     'start_date','end_date', 'description',
                                     'award',
                                     'publication', 
                                     'additional_info', 
                                     'skill'])]
data_valid.head()

,user_id,name,value_char,value_timestamp
0,3.257237e+11,name,Freelance Copy Editor of 100,NaT
1,1.987262e+11,admit_year,None,2016-01-01
2,1.987262e+11,grad_year,present,NaT
3,1.987262e+11,admit_year,None,2015-01-01
4,1.987262e+11,admit_year,None,2012-01-01


In [8]:
def clean_text(prefix=None):
    '''
    Cleans text data by:
    1.  force lowercase
    2.  remove non-ascii chars
    3.  standardize whitespace
    4.  remove digits
    5.  remove control characters
    6.  remove URL patterns
    '''
    df = pd.DataFrame()
    
    try:
        df = data_valid[prefix].dropna().map(lambda x: "".join(i for i in x.strip().lower() if ord(i)<128))
    except UnicodeDecodeError:
        print(UnicodeDecodeError)
        df = data_valid[prefix].dropna().map(lambda x: x.strip().lower())

        #     except Exception:
#         print(Exception)
#     finally:
#         data[prefix]= data[prefix].map(lambda x: x.lower())

    url_pattern = "((http|ftp|https):\/\/)?[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?"

    re_URL = re.compile(url_pattern)
#     re_TAG = re.compile("(<[phl]>)", re.IGNORECASE)
    re_WS = re.compile("/[^\S\n]/")
#     re_DIGIT = re.compile("\d")
    re_CTRL = re.compile("[\x00-\x11\x03-\x1F]+")
    re_HI = re.compile("[\x80-\xFF]+")
    re_NWC = re.compile("[!;<>?{}\/~`#=@#$%^&*()_+]")
    
    df = df.map(lambda x: re_HI.sub(' ', x))
    df = df.map(lambda x: re_CTRL.sub(' ', x))
    df = df.map(lambda x: re_URL.sub(' ', x))
#     data[prefix] = data[prefix].map(lambda x: re_DIGIT.sub(' ', x))
    df = df.map(lambda x: re_WS.sub(' ', x))        
    df = df.map(lambda x: re_NWC.sub(' ', x))
    df = df.map(lambda x: x.strip())
    
    return df

In [9]:
clean_text('value_char').head()

0             freelance copy editor of 100
2                                  present
5    research and commercialisation intern
6             hold full uk driving license
8            volunteer  campsite attendant
Name: value_char, dtype: object

In [10]:
data_clean = pd.DataFrame()
data_clean['value_char'] = clean_text('value_char')
data_clean.head()

,value_char
0,freelance copy editor of 100
2,present
5,research and commercialisation intern
6,hold full uk driving license
8,volunteer campsite attendant


In [11]:
# data_valid = data_valid.reset_index(drop=True)

In [12]:
# ## Tokenization

# # initialize the tokenizer

# tokenizer = nltk.tokenize.PunktSentenceTokenizer()

# # tokenize data

# data_clean['value_char'] = data_clean['value_char'].map(lambda x: tokenizer.tokenize(x.strip()))
# data_clean.head()

In [13]:
# word_tokenizer = nltk.tokenize.word_tokenize()
# data_clean['value_char'] = data_clean['value_char'].map(lambda x: nltk.tokenize.word_tokenize(x))

In [14]:
# ## Stemming

# # initialize the stemmer

# stemmer = nltk.stem.PorterStemmer()
# # data_clean['value_char'].map(lambda x: (i for i in x))

In [15]:
# lemmatize = nltk.WordNetLemmatizer

In [16]:
# cv_headlines = data[data['name'] == 'headline'].reset_index()[['user_id','value_char']]
# cv_degrees = data[data['name'] == 'degree'].reset_index()[['user_id','value_char']]
# cv_schools = data[data['name'] == 'school'].reset_index()[['user_id','value_char']]
# cv_locales = data[data['name'] == 'locale'].reset_index()[['user_id','value_char']]
# cv_summaries = data[data['name'] == 'summary'].reset_index()[['user_id','value_char']]

In [17]:
stopWords = stopwords.words('english')
tokenizer_prefs = {
    'tokenizer' : nltk.tokenize.PunktSentenceTokenizer(),
#     'token_format' : 'stem',
    'spell_correct' : False,
    'np_extract': None,
    'pos_tagger': None,
    'analyzer': None,
    'classifier': None, 
    'clean_html': False
}

In [18]:
def create_blob(prefix, **kwargs):
    
    tokenizer = kwargs['tokenizer']
    pos_tagger = kwargs['pos_tagger']
    analyzer = kwargs['analyzer']
    classifier = kwargs['classifier']
    np_extract = kwargs['np_extract']
    
    blob = data_clean[prefix].map(lambda l: TextBlob(l,
                                          tokenizer=tokenizer,
                                           np_extractor=np_extract,
                                           pos_tagger=pos_tagger,
                                           analyzer=analyzer,
                                           classifier=classifier))
    return blob

In [19]:
data_blobs = pd.DataFrame()
data_blobs['value_char'] = create_blob('value_char', **tokenizer_prefs)
data_blobs.value_char[0:10]

0     (f, r, e, e, l, a, n, c, e,  , c, o, p, y,  , ...
2                                 (p, r, e, s, e, n, t)
5     (r, e, s, e, a, r, c, h,  , a, n, d,  , c, o, ...
6     (h, o, l, d,  , f, u, l, l,  , u, k,  , d, r, ...
8     (v, o, l, u, n, t, e, e, r,  ,  , c, a, m, p, ...
9                                    (s, k, i, l, l, s)
10    (l, i, g, h, t, -, s, c, a, t, t, e, r, i, n, ...
13    (p, o, s, t, -, d, o, c, t, o, r, a, l,  , r, ...
14    (t, e, a, m,  , l, e, a, d, e, r,  ,  ,  , n, ...
15           (r, e, l, i, e, f,  , m, a, n, a, g, e, r)
Name: value_char, dtype: object

In [20]:
def tokenize_sentences(prefix):
#     tokenizer = kwargs['tokenizer']
#     normalizer = kwargs['token_format']

    # tokenize the document into sentences from blob object
    sentences = data_blobs[prefix].map(lambda s: s.sentences)

    return sentences

In [21]:
data_sentence_tokenized = pd.DataFrame()
data_sentence_tokenized['value_char'] = tokenize_sentences('value_char')
data_sentence_tokenized.value_char[0:10]

0     [(f, r, e, e, l, a, n, c, e,  , c, o, p, y,  ,...
2                               [(p, r, e, s, e, n, t)]
5     [(r, e, s, e, a, r, c, h,  , a, n, d,  , c, o,...
6     [(h, o, l, d,  , f, u, l, l,  , u, k,  , d, r,...
8     [(v, o, l, u, n, t, e, e, r,  ,  , c, a, m, p,...
9                                  [(s, k, i, l, l, s)]
10    [(l, i, g, h, t, -, s, c, a, t, t, e, r, i, n,...
13    [(p, o, s, t, -, d, o, c, t, o, r, a, l,  , r,...
14    [(t, e, a, m,  , l, e, a, d, e, r,  ,  ,  , n,...
15         [(r, e, l, i, e, f,  , m, a, n, a, g, e, r)]
Name: value_char, dtype: object

In [114]:
def tokenize_words(prefix):
#     tokenizer = kwargs['tokenizer']
#     normalizer = kwargs['token_format']

    # tokenize each sentence into words
    # trim token whitespaces
    # eliminate tokens of character length 1
    #words = self.data[prefix].map(lambda w: w.strip().tokens if len(w)>1 else None)

    words = data_sentence_tokenized[prefix].map(lambda l: map(lambda w: w.strip().words if len(w)>1 else None, l))

    return words

In [115]:
# data_word_tokenized = pd.DataFrame()
data_word_tokenized = tokenize_words('value_char')

In [116]:
def normalize_data(data_word_tokenized):
#     tokenizer = kwargs['tokenizer']
#     normalizer = kwargs['token_format']
#     spelling = kwargs['spell_correct']
    
    data_normalized = data_word_tokenized.map(lambda l: map(lambda w: w.singularize() if not isinstance(w, type(None)) and len(w)>1 else None, l))
#     data_normalized = data_normalized.dropna()
    
    # filter out 'bad' words, normalize good ones
    # w if w not in self.stopWords else wl.remove(w)
#     data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w in stopWords and not isinstance(w, type(None)) else w, wl), l))
#     data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w == '\'s' and not isinstance(w, type(None) else w, wl), l))
#     data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w == '\'d' and not isinstance(w, type(None) else w, wl), l))
#     data_normalized = data_normalized.dropna()

     # remove tokens with length 1
#     ree = re.compile(r'(\'\w)')
#     rlen = len(data_normalized)
#     tmp = data_normalized.copy()
# #     tmp.reset_index(drop=True)
#     for indx in tmp.index:
#          wl_coll = list()
#          for i in normalize(data_word_tokenized,**tokenizer_prefs):
#             for j in i:
#                 if not isinstance(j, type(None)):
#                     for k in j:
#                         if not isinstance(k, type(None)):
#                             if re.match(ree, i):
#                                 ree.sub('', i)
#                             if len(i.strip().strip('.').strip(',')) > 1:
#                                 wl.append((i))
#                     wl_coll.append(WordList(wl))
#             data_normalized[indx] = wl_col
#             del tmp

    # stemming
#     data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: nltk.stemmer.stem(w) if w in wl and not isinstance(w, types.NoneType) else wl.remove(w), wl), l))

#     data_word_tokenized= tokenize_words('value_char', **tokenizer_prefs)
    
    return data_normalized

In [117]:
data_normalized = normalize_data(data_word_tokenized)

In [118]:
# if not isinstance(data_normalized, type(None)):
#     for i in data_normalized:
#         if not isinstance(i, type(None)):
#             for j in i:
#                 if not isinstance(j, type(None)):
#                     for k in j:
#                         if not isinstance(k, type(None)) and k not in stopWords and k != '\'s' and k != '\'d':
#                             print(k)

In [119]:
def build_vocab(data_normalized, target):
    vocab = set()
    if not isinstance(data_normalized, type(None)):
        for token in data_normalized:
            if not isinstance(token, type(None)):
                for sentence in token:
                    if not isinstance(sentence, type(None)):
                        for word in sentence:
                            if not isinstance(word, type(None)) and word not in stopWords and word != '\'s' and word != '\'d':
                                vocab.add(word)
            
    if target:
        w2i = {w: np.int32(i+2) for i, w in enumerate(vocab)}
        w2i['<s>'], w2i['</s>'] = np.int32(0), np.int32(1)
    else:
        w2i = {w: np.int32(i) for i, w in enumerate(vocab)}

    return w2i

In [55]:
w2i = build_vocab(data_normalized, True)

In [65]:
w2i

{'hypothesi': 2,
 'etching': 3,
 'orthopedist': 4,
 'premiere-based': 5,
 'perday': 6,
 'andposition': 7,
 'outlay': 8,
 'lorry': 10,
 'gallup': 36256,
 'semisolid': 11,
 '2013tinatus': 12,
 'sister': 13,
 'became': 14,
 'geometryal-qud': 15,
 'vmsimplementing': 16,
 'sh': 17,
 'rutgers-the': 18,
 'dorotocephala': 19,
 'windowsapplication': 20,
 '2015led': 21,
 '850,856': 27051,
 'othersfocused': 22,
 'sasproject': 8931,
 '429': 23,
 'benefitsresource': 24,
 'data-cabling': 25,
 'developmentfinancial': 26,
 'thesis-analyzed': 27053,
 'reportsquality': 28,
 'steward': 30,
 'aif': 33,
 'pyrolysi': 44143,
 'analysisconsultantsagile': 34,
 'r2application': 36,
 'syspro': 42,
 'deliving': 38,
 't2': 39,
 'compounddissolution': 41,
 'whilstbeing': 9407,
 'dorado': 9,
 '2764': 46675,
 'emphasize': 43,
 "'student": 44,
 'arts-political': 45,
 'stagecommunicating': 49,
 'severely': 48,
 'catalua': 8937,
 'best-of': 50,
 'instance': 51,
 'financeuniversity': 8938,
 'servicesprofessional': 52,
 '

In [102]:
def encode(data_normalized, w2i):
    encoded_sentence = []
    if not isinstance(data_normalized, type(None)):
        for token in data_normalized:
            if not isinstance(token, type(None)):
                for sentence in token:
                    if not isinstance(sentence, type(None)):
                        for w in sentence:
                            try:
                                encoded_sentence.append(w2i[w])
                            except Exception:
                                pass
    return encoded_sentence

In [122]:
def load_data(data_normalized, vocab=None, w2i=None, target=True):
    if vocab is None and w2i is None:
        w2i = build_vocab(data_normalized, target)

    s = []
    if not isinstance(data_normalized, type(None)):
        for token in data_normalized:
            if not isinstance(token, type(None)):
                for sentence in token:
                    if not isinstance(sentence, type(None)):
                        for w in sentence:
                            s.append(w)
        if target:
            s = ['<s>'] + s + ['</s>']
        enc = encode(s, w2i)
        data.append(enc)
    i2w = {i: w for w, i in w2i.items()}
    return data, w2i, i2w

In [120]:
data_all = []
if not isinstance(data_normalized, type(None)):
    for token in data_normalized:
        if not isinstance(token, type(None)):
            for sentence in token:
                if not isinstance(sentence, type(None)):
                    for s in sentence:
                        data_all.append(s)

In [121]:
data_all

['freelance',
 'copy',
 'editor',
 'of',
 '100',
 'research',
 'and',
 'commercialisation',
 'intern',
 'hold',
 'full',
 'uk',
 'driving',
 'license',
 'volunteer',
 'campsite',
 'attendant',
 'light-scattering',
 'technique',
 'inkjet',
 'printing',
 'nmr',
 'size-exclusion',
 'chromatograhy',
 'uv-visible',
 'spectroscopy',
 'fluorescence',
 'post-doctoral',
 'research',
 'associate',
 'team',
 'leader',
 'night',
 'manager',
 'relief',
 'manager',
 'supervisor',
 'duty',
 'manager',
 'assistant',
 'manager',
 'postdoctoral',
 'research',
 'assistant',
 'subharmonic',
 'function',
 'on',
 'graph',
 'differential',
 'equation',
 'project',
 'within',
 'the',
 'course',
 'of',
 'programming',
 'expertise',
 'and',
 'consulting',
 'delivering',
 'result',
 'preparing',
 'concise',
 'and',
 'logically-written',
 'material',
 'ability',
 'to',
 'organize',
 'and',
 'communicate',
 'idea',
 'effectively',
 'in',
 'oral',
 'presentation',
 'to',
 'small',
 'and',
 'large',
 'group',
 'meet

In [110]:
a = '''sdasd asdasd sdasdas'''

s = a.split()

s = ['<s>'] + s + ['</s>']
s

['sdasd', 'asdasd', 'sdasdas']

In [ ]:
# def build_vocab(data_word_tokenized, target):
#     vocab = set()
#     for token in data_word_tokenized:
#         for sent in token:
#             if not isinstance(sent, type(None)):
#                 for word in sent:
#                     vocab.add(word)
            
#     if target:
#         w2i = {w: np.int32(i+2) for i, w in enumerate(vocab)}
#         w2i['<s>'], w2i['</s>'] = np.int32(0), np.int32(1)
#     else:
#         w2i = {w: np.int32(i) for i, w in enumerate(vocab)}

#     return w2i

In [123]:
# data_normalized = data_word_tokenized.map(lambda l: map(lambda wl: map(lambda w: nltk.stem.PorterStemmer.NLTK_EXTENSIONS(w) if w in wl and not isinstance(w, type(None)) else wl.remove(w), wl), l))

In [131]:
ree = re.compile(r'(\'\w)')
wl = list()

for i in data_normalized:
    for j in i:
        if not isinstance(j, type(None)):
            for k in j:
                for l in k:
            

TypeError: 'str' object is not callable

In [ ]:
ree = re.compile(r'(\'\w)')
wl = list()

for i in normalize(data_word_tokenized,**tokenizer_prefs):
    for j in i:
        if not isinstance(j, type(None)):
            for k in j:
                if not isinstance(k, type(None)):
                    if re.match(ree, k):
                        ree.sub('', k)
                    if len(k.strip().strip('.').strip(',')) > 1:
                        wl.append((k))
            

In [101]:
def removeNoneTypes(lst):
    return [i for i in lst if type(i) is not type(None)]

In [105]:
def normalize(data_word_tokenized, **kwargs):
    tokenizer = kwargs['tokenizer']
    normalizer = kwargs['token_format']
    spelling = kwargs['spell_correct']
    
    data_normalized = data_word_tokenized.map(lambda l: map(lambda wl: removeNoneTypes(wl), l))
    data_normalized = data_normalized.map(lambda l: map(lambda w: w.singularize(), l))
    
    # filter out 'bad' words, normalize good ones
    # w if w not in self.stopWords else wl.remove(w)
    data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w in stopWords else w, wl), l))
    data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w == '\'s' else w, wl), l))
    data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w == '\'d' else w, wl), l))
 
     # remove tokens with length 1
#     ree = re.compile(r'(\'\w)')
#     rlen = len(data_normalized)
#     tmp = data_normalized.copy()
# #     tmp.reset_index(drop=True)
#     for indx in tmp.index:
         wl_coll = list()
         for i in normalize(data_word_tokenized,**tokenizer_prefs):
            for j in i:
                if not isinstance(j, type(None)):
                    for k in j:
                        if not isinstance(k, type(None)):
                            if re.match(ree, i):
                                ree.sub('', i)
                            if len(i.strip().strip('.').strip(',')) > 1:
                                wl.append((i))
                    wl_coll.append(WordList(wl))
            data_normalized[indx] = wl_col
            del tmp

    # stemming
    data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: nltk.stemmer.stem(w) if w in wl and not isinstance(w, types.NoneType) else wl.remove(w), wl), l))

    data_word_tokenized= tokenize_words('value_char', **tokenizer_prefs)
    
    return data_word_tokenized

In [108]:
stopWords = stopwords.words('english')
tokenizer  =   RegexpTokenizer(pattern=r'\w+')
stemmer    =   nltk.stem.PorterStemmer.NLTK_EXTENSIONS
lemmatize  =   nltk.WordNetLemmatizer()

In [107]:
normalize(data_word_tokenized,**tokenizer_prefs)

0        <map object at 0x7fbaa7e44550>
1        <map object at 0x7fbaa7e445c0>
2        <map object at 0x7fbaa7e44630>
3        <map object at 0x7fbaa7e446d8>
4        <map object at 0x7fbaa7e44780>
6        <map object at 0x7fbaa7e44828>
7        <map object at 0x7fbaa7e448d0>
8        <map object at 0x7fbaa7e44978>
9        <map object at 0x7fbaa7e44a20>
10       <map object at 0x7fbaa7e44ac8>
11       <map object at 0x7fbaa7e44b70>
12       <map object at 0x7fbaa7e44c18>
13       <map object at 0x7fbaa7e44cc0>
14       <map object at 0x7fbaa7e44d68>
18       <map object at 0x7fbaa7e44e10>
19       <map object at 0x7fbaa7e44eb8>
21       <map object at 0x7fbaa7e44f60>
22       <map object at 0x7fbaa7e46048>
23       <map object at 0x7fbaa7e460f0>
24       <map object at 0x7fbaa7e46198>
25       <map object at 0x7fbaa7e46240>
29       <map object at 0x7fbaa7e462e8>
30       <map object at 0x7fbaa7e46390>
31       <map object at 0x7fbaa7e46438>
32       <map object at 0x7fbaa7e464e0>


In [108]:
ree = re.compile(r'(\'\w)')
wl = list()

for i in normalize(data_word_tokenized,**tokenizer_prefs):
    for j in i:
        if not isinstance(j, type(None)):
            for k in j:
                if not isinstance(k, type(None)):
                    if re.match(ree, k):
                        ree.sub('', k)
                    if len(k.strip().strip('.').strip(',')) > 1:
                        wl.append((k))
            

In [109]:
wl

['mitcham',
 'guest',
 'lecturer',
 'crewe',
 'pencari',
 'kerja',
 'an',
 'ambitious',
 'and',
 'hardworking',
 'individual',
 'who',
 'is',
 'motivated',
 'by',
 'challenge',
 'and',
 'is',
 'passionate',
 'to',
 'be',
 'successful',
 'inventor',
 'on',
 'four',
 'patents',
 'excellent',
 'communicator',
 'strong',
 'planning',
 'organisational',
 'problem',
 'solving',
 'skills',
 'with',
 'the',
 'ability',
 'to',
 'successfully',
 'analyse',
 'and',
 'assimilate',
 'large',
 'amounts',
 'of',
 'complex',
 'and',
 'disparate',
 'information',
 'good',
 'time',
 'management',
 'enjoys',
 'working',
 'under',
 'pressure',
 'and',
 'to',
 'deadlines',
 'either',
 'individually',
 'or',
 'as',
 'part',
 'of',
 'team',
 'conversational',
 'german',
 'part',
 'time',
 'lecturing',
 'and',
 'lab',
 'demonstrator',
 'project',
 'evaluation',
 'and',
 'responsible',
 'innovation',
 'intern',
 'computer',
 'skills-',
 'good',
 'knowledge',
 'of',
 'it',
 'and',
 'confident',
 'in',
 'using',

In [ ]:


def normalize_data(**kwargs):
    tokenizer = kwargs['tokenizer']
    normalizer = kwargs['token_format']
    spelling = kwargs['spell_correct']
    
#     data = pd.DataFrame(['asc','asda','asdasdasd'], columns=['value_char'])
    
    # singularize tokens
#     data = data[prefix].map(lambda l: map(lambda w: w.singularize(), l))

    # Spell correct flag
    # REALLY SHOULD NEVER BE USED
#     if spelling:
#         print("Spell Correction Invoked.....")
#         data[prefix] = data[prefix].map(lambda l: map(lambda wl: map(lambda w: w.correct(), wl), l))
#         print(data[prefix].map(lambda l: map(lambda w: type(w), l)))

    # filter out 'bad' words, normalize good ones
    # w if w not in self.stopWords else wl.remove(w)
    aa = cleanestes.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w in stopWords else w, wl), l))
    aa = aa.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w == '\'s' else w, wl), l))
    aa = aa.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w == '\'d' else w, wl), l))

    # remove tokens with length 1
#     ree = re.compile(r'(\'\w)')
#     rlen = len(data)
#     tmp = data[prefix].copy()
#     for index in range(0,rlen):
#         wl_coll = list()
#         for lst in tmp[index]:
#             wl = list()
#             for word in lst:
#                 if not isinstance(word, types.NoneType):
#                     if re.match(ree, word):
#                         ree.sub('', word)
#                     if len(word.strip().strip('.').strip(',')) > 1:
#                         wl.append((word))
#             wl_coll.append(WordList(wl))
#         data[index] = wl_coll
#     del tmp

    # remove via regexp c'c pattern

    # Stemming or lemmatization of tokens    
    if normalizer == 'stem':
        aa = aa.map(lambda l: map(lambda wl: map(lambda w: stemmer.stem(w) if w in wl and not isinstance(w, types.NoneType) else wl.remove(w), wl), l))
#     elif normalizer == 'lemma':
#         data[prefix] = data[prefix].map(lambda l: map(lambda wl: map(lambda w: w.lemmatize(), wl), l))
#     elif normalizer == 'None':
#         pass

    aa = aa.map(lambda l: map(lambda wl: map(Word, wl), l))
#     data[prefix] = data[prefix].map(lambda l: map(WordList, l))
    
    return aa

In [ ]:
normalize_data(**tokenizer_prefs)